<a href="https://colab.research.google.com/github/KashyapCKotak/AI-ML-experiments/blob/master/SeeInDark_Keras_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import os

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)



print('Dowloading Sony subset... (25GB)')
download_file_from_google_drive('10kpAcvldtcb9G2ze5hTcF1odzu4V_Zvh', './Sony.zip')

#print('Dowloading Fuji subset... (52GB)')
#download_file_from_google_drive('12hvKCjwuilKTZPe9EZ7ZTb-azOmUA3HT', 'dataset/Fuji.zip')

os.system('unzip ./Sony.zip -d dataset')
#os.system('unzip dataset/Fuji.zip -d dataset')


Dowloading Sony subset... (25GB)


In [0]:
import shutil
shutil.rmtree('result_Sony', ignore_errors=True, onerror=None)

In [0]:
os.system('unzip ./Sony.zip -d dataset')
#os.system('unzip dataset/Fuji.zip -d dataset')

0

**Main Train Code:**
=================================

In [32]:
!pip install rawpy

In [0]:
from __future__ import division
import os, time, scipy.io
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import rawpy
import glob
import keras

In [0]:
import shutil
shutil.rmtree('result_Sony', ignore_errors=True, onerror=None)

input_dir = '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/short/'
gt_dir = '/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/long/'
os.mkdir('result_Sony')
checkpoint_dir = './result_Sony/'
result_dir = './result_Sony/'

In [0]:
train_fns = glob.glob(gt_dir + '0*.ARW')
###
# 0* is for Training images. 1 for Test and 2 for Validation
###
train_ids = [int(os.path.basename(train_fn)[0:5]) for train_fn in train_fns]

In [0]:
ps = 512  # just a random patch size for training
save_freq = 500

In [0]:
DEBUG = 0
if DEBUG == 1:
    save_freq = 2
    train_ids = train_ids[0:5]

In [0]:
def lrelu(x): # not used. Used Kears LeakyReLU with alpha 0.2 instead
    return tf.maximum(x * 0.2, x)

In [0]:
def upsample_and_concat(x1,x2,output_channels,in_channels):
  #x1=vars[0]
  #x2=vars[1]
  #output_channels=vars[3]
  #in_channels=vars[4]
  
  pool_size = 2
  deconv_filter = tf.Variable(tf.truncated_normal([pool_size, pool_size, output_channels, in_channels], stddev=0.02))
  deconv = tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2), strides=[1, pool_size, pool_size, 1])
  #deconv=keras.layers.Conv2DTranspose(filters=tf.shape(x2),kernel_size=deconv_filter,strides=[1, pool_size, pool_size, 1])(x1)

  deconv_output = tf.concat([deconv, x2], 3)
  deconv_output.set_shape([None, None, None, output_channels * 2])

  return deconv_output

In [0]:
def Depth_to_space_tf(input):
  return tf.depth_to_space(input, 2)

def build_model():
  inputs=keras.layers.Input(shape=(None, None, 4))
  #definition: slim.conv2d(inputs,number of outputs, kernel shape)
  #definition: slim.max_pool2d(inputsconv2d_transpose ,kernel_size, padding)
  #conv1 = slim.conv2d(input, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_1')
  conv1=keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),name='conv1_1')(inputs)
  conv1=keras.layers.LeakyReLU(alpha=0.2,name='conv1_1_relu')(conv1)
  #conv1 = slim.conv2d(conv1, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_2')
  conv1=keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),name='conv1_2')(conv1)
  conv1=keras.layers.LeakyReLU(alpha=0.2,name='conv1_2_relu')(conv1)
  #pool1 = slim.max_pool2d(conv1, [2, 2], padding='SAME')
  pool1=keras.layers.MaxPooling2D(pool_size=(2,2),padding="same",name='pool1')(conv1)

  #conv2 = slim.conv2d(pool1, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_1')
  conv2=keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),name='conv2_1')(pool1)
  conv2=keras.layers.LeakyReLU(alpha=0.2,name='conv2_1_relu')(conv2)
  #conv2 = slim.conv2d(conv2, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_2')
  conv2=keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),name='conv2_2')(conv2)
  conv2=keras.layers.LeakyReLU(alpha=0.2,name='conv2_2_relu')(conv2)
  #pool2 = slim.max_pool2d(conv2, [2, 2], padding='SAME')
  pool2=keras.layers.MaxPooling2D(pool_size=(2,2),padding="same",name='pool2')(conv2)

  #conv3 = slim.conv2d(pool2, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_1')
  conv3=keras.layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),name='conv3_1')(pool2)
  conv3=keras.layers.LeakyReLU(alpha=0.2,name='conv3_1_relu')(conv3)
  #conv3 = slim.conv2d(conv3, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_2')
  conv3=keras.layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),name='conv3_2')(conv3)
  conv3=keras.layers.LeakyReLU(alpha=0.2,name='conv3_2_relu')(conv3)
  #pool3 = slim.max_pool2d(conv3, [2, 2], padding='SAME')
  pool3=keras.layers.MaxPooling2D(pool_size=(2,2),padding="same",name='pool3')(conv3)

  #conv4 = slim.conv2d(pool3, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_1')
  conv4=keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),name='conv4_1')(pool3)
  conv4=keras.layers.LeakyReLU(alpha=0.2,name='conv4_1_relu')(conv4)
  #conv4 = slim.conv2d(conv4, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_2')
  conv4=keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),name='conv4_2')(conv4)
  conv4=keras.layers.LeakyReLU(alpha=0.2,name='conv4_2_relu')(conv4)
  #pool4 = slim.max_pool2d(conv4, [2, 2], padding='SAME')
  pool4=keras.layers.MaxPooling2D(pool_size=(2,2),padding="same",name='pool4')(conv4)

  #conv5 = slim.conv2d(pool4, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_1')
  conv5=keras.layers.Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),name='conv5_1')(pool4)
  conv5=keras.layers.LeakyReLU(alpha=0.2,name='conv5_1_relu')(conv5)
  #conv5 = slim.conv2d(conv5, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_2')
  conv5=keras.layers.Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),name='conv5_2')(conv5)
  conv5=keras.layers.LeakyReLU(alpha=0.2,name='conv5_2_relu')(conv5)

  #up6 = upsample_and_concat(conv5, conv4, 256, 512)
  up6=keras.layers.core.Lambda(upsample_and_concat,arguments={'x2':conv4,'output_channels':256,'in_channels':512},name='upsample_concat_1')(conv5)
  #conv6 = slim.conv2d(up6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_1')
  conv6=keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),name='conv6_1')(up6)
  conv6=keras.layers.LeakyReLU(alpha=0.2,name='conv6_1_relu')(conv6)
  #conv6 = slim.conv2d(conv6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_2')
  conv6=keras.layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),name='conv6_2')(conv6)
  conv6=keras.layers.LeakyReLU(alpha=0.2,name='conv6_2_relu')(conv6)

  #up7 = upsample_and_concat(conv6, conv3, 128, 256)
  up7=keras.layers.core.Lambda(upsample_and_concat,arguments={'x2':conv3,'output_channels':128,'in_channels':256},name='upsample_concat_2')(conv6)
  #conv7 = slim.conv2d(up7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_1')
  conv7=keras.layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),name='conv7_1')(up7)
  conv7=keras.layers.LeakyReLU(alpha=0.2,name='conv7_1_relu')(conv7)
  #conv7 = slim.conv2d(conv7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_2')
  conv7=keras.layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),name='conv7_2')(conv7)
  conv7=keras.layers.LeakyReLU(alpha=0.2,name='conv7_2_relu')(conv7)

  #up8 = upsample_and_concat(conv7, conv2, 64, 128)
  up8=keras.layers.core.Lambda(upsample_and_concat,arguments={'x2':conv2,'output_channels':64,'in_channels':128},name='upsample_concat_3')(conv7)
  #conv8 = slim.conv2d(up8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_1')
  conv8=keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),name='conv8_1')(up8)
  conv8=keras.layers.LeakyReLU(alpha=0.2,name='conv8_1_relu')(conv8)
  #conv8 = slim.conv2d(conv8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_2')
  conv8=keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),name='conv8_2')(conv8)
  conv8=keras.layers.LeakyReLU(alpha=0.2,name='conv8_2_relu')(conv8)

  #up9 = upsample_and_concat(conv8, conv1, 32, 64)
  up9=keras.layers.core.Lambda(upsample_and_concat,arguments={'x2':conv1,'output_channels':32,'in_channels':64},name='upsample_concat_4')(conv8)
  #conv9 = slim.conv2d(up9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_1')
  conv9=keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),name='conv9_1')(up9)
  conv9=keras.layers.LeakyReLU(alpha=0.2,name='conv9_1_relu')(conv9)
  #conv9 = slim.conv2d(conv9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_2')
  conv9=keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),name='conv9_2')(conv9)
  conv9=keras.layers.LeakyReLU(alpha=0.2,name='conv9_2_relu')(conv9)

  #conv10 = slim.conv2d(conv9, 12, [1, 1], rate=1, activation_fn=None, scope='g_conv10')
  conv10=keras.layers.Conv2D(filters=12,kernel_size=(1,1),strides=(1,1),name='conv10')(conv9)
  #no activation function for this last layer

  predictions = keras.layers.core.Lambda(Depth_to_space_tf,name='depth_to_space')(conv10)
  model = keras.models.Model(inputs=inputs, outputs=predictions)
  
  return model

In [0]:
def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    #print("im shape:",im.shape)
    #print("im:")
    #print(im)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level and normalise with 16383 
    ##
    # Seems 16383 is the max value due to a 14 bit depth. can be replaced with 
    ##
    #print("after normalisation")
    #print(im)

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]
    #print("after expand dims:%s ,H:%d ,W:%d " % (img_shape,H,W))

    ###
    # single bayer pixel format:
    # 0:R 1:G
    # 2:G 3:B
    ###
    out = np.concatenate((im[0:H:2, 0:W:2, :], # get the 0th Red as per above format
                          im[0:H:2, 1:W:2, :], # get the 1st Green
                          im[1:H:2, 1:W:2, :], # get the 2nd Green
                          im[1:H:2, 0:W:2, :]), axis=2) # get the 3rd Green
    #print("out.shape:",out.shape)
    #print(out)
    return out


In [0]:
#sess = tf.Session()
in_image = tf.placeholder(tf.float32, [None, None, None, 4])
gt_image = tf.placeholder(tf.float32, [None, None, None, 3])
#out_image = network(in_image)

In [0]:
#G_loss = tf.reduce_mean(tf.abs(out_image - gt_image))
t_vars = tf.trainable_variables()
#lr = tf.placeholder(tf.float32)
#G_opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss)
#saver = tf.train.Saver()
def reduced_mean(y_true,y_pred):
  return tf.reduce_mean(tf.abs(y_pred - y_true))

In [0]:
#ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
#if ckpt:
#    print('loaded ' + ckpt.model_checkpoint_path)
#    saver.restore(sess, ckpt.model_checkpoint_path)
###
# above commented as not needed can be uncommented
###


In [0]:
gt_images = [None] * 6000
input_images = {}
input_images['300'] = [None] * len(train_ids)
input_images['250'] = [None] * len(train_ids)
input_images['100'] = [None] * len(train_ids)

In [0]:
#? g_loss = np.zeros((5000, 1))

In [0]:
allfolders = glob.glob('./result/*0')
lastepoch = 0
for folder in allfolders:
    lastepoch = np.maximum(lastepoch, int(folder[-4:]))
    
###
# The above code is used to infer the last completed epoch
# ( may be in case a previously saved model is restored)
###

In [0]:
learning_rate = 0.0001

In [59]:
train_ids[0:10]

[90, 91, 92, 72, 73, 75, 76, 78, 81, 83]

In [60]:
len(train_ids)

161

In [0]:
train_id = 1
in_files = glob.glob(input_dir + '%05d_00*.ARW' % train_id) # get all the images path with pattern 0*train_id
#print(in_files)
in_path = in_files[np.random.randint(0, len(in_files))] # get any one image randomly
#print(in_path)
in_fn = os.path.basename(in_path) # get only the full image name
#print(in_fn)

gt_files = glob.glob(gt_dir + '%05d_00*.ARW' % train_id) # get the ground truth image path(s) (only 1 may exist. hence select [0]th below)
#print(gt_files)
gt_path = gt_files[0] # get the first one (as only 1 gt image should exist)
#print(gt_path)
gt_fn = os.path.basename(gt_path) # get only the full image name for gt image
#print(gt_fn)
in_exposure = float(in_fn[9:-5]) # get the exposure for input image
#print(in_exposure)
gt_exposure = float(gt_fn[9:-5]) # get the exposure for gt image
#print(gt_exposure)
ratio = min(gt_exposure / in_exposure, 300)
print("ratio:",ratio)
########################
gt_raw = rawpy.imread('/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/long/00001_00_10s.ARW')
print(gt_raw.raw_pattern)
print(gt_raw)
im = gt_raw.postprocess(use_camera_wb=True,
                        half_size=False,
                        no_auto_bright=True, output_bps=16)
print(im.shape)
print(im[0:2,:,:])
gt_images = np.expand_dims(np.float32(im / 65535.0),
        axis=0)
print("gt_image shape",gt_images.shape)
print(gt_images[:,0:2,:,:])
############################
raw = rawpy.imread('/content/drive/My Drive/Data/SeeInDarkDataset/dataset/Sony/short/00001_00_0.04s.ARW')
packed_input_image=pack_raw(raw)
input_img = np.expand_dims(packed_input_image, axis=0) * ratio
print("input_image shape",input_img.shape)
print(input_img[:,0:2,:,:])


In [69]:
model=build_model()
print(model.summary())
keras.utils.vis_utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
model.compile(optimizer='adam',loss=reduced_mean)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, None, None, 4)     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, None, None, 32)    1184      
_________________________________________________________________
conv1_1_relu (LeakyReLU)     (None, None, None, 32)    0         
_________________________________________________________________
conv1_2 (Conv2D)             (None, None, None, 32)    9248      
_________________________________________________________________
conv1_2_relu (LeakyReLU)     (None, None, None, 32)    0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, None, None, 32)    0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, None, None, 64)    18496     
__________

W0726 11:03:46.557630 139637959735168 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
for epoch in range(lastepoch, 4001):
    #if os.path.isdir('result/%04d' % epoch):
    #    continue
    ###
    # used for skipping the current epoch if the checkpoint directory exists
    ###
    cnt = 0
    if epoch > 2000:
        learning_rate = 1e-5

    for ind in np.random.permutation(len(train_ids)):

        # get the path from image id

        train_id = train_ids[ind]
        in_files = glob.glob(input_dir + '%05d_00*.ARW' % train_id) # get all the images path with pattern 0*train_id
        in_path = in_files[np.random.randint(0, len(in_files))] # get any one image randomly
        in_fn = os.path.basename(in_path) # get only the full image name

        gt_files = glob.glob(gt_dir + '%05d_00*.ARW' % train_id) # get the ground truth image path(s) (only 1 may exist. hence select [0]th below)
        gt_path = gt_files[0] # get the first one (as only 1 gt image should exist)
        gt_fn = os.path.basename(gt_path) # get only the full image name for gt image
        in_exposure = float(in_fn[9:-5]) # get the exposure for input image
        gt_exposure = float(gt_fn[9:-5]) # get the exposure for gt image
        ratio = min(gt_exposure / in_exposure, 300) # get the amplification ratio

        st = time.time()
        cnt += 1

        if input_images[str(ratio)[0:3]][ind] is None: # if image is not loaded (first epoch), load it
            raw = rawpy.imread(in_path)
            input_images[str(ratio)[0:3]][ind] = \
                np.expand_dims(pack_raw(raw), axis=0) * ratio # pack the bayer image in 4 channels of RGBG

            gt_raw = rawpy.imread(gt_path)
            im = gt_raw.postprocess(use_camera_wb=True,
                                    half_size=False,
                                    no_auto_bright=True, output_bps=16)
            gt_images[ind] = np.expand_dims(np.float32(im / 65535.0),axis=0) # divide by 65535 to normalise (scale between 0 and 1)

        # crop

        H = input_images[str(ratio)[0:3]][ind].shape[1] # get the image height (number of rows)
        W = input_images[str(ratio)[0:3]][ind].shape[2] # get the image width (number of columns)

        xx = np.random.randint(0, W - ps) # get a random number in W-ps (W-512)
        yy = np.random.randint(0, H - ps) # get a random number in H-ps (H-512)
        input_patch = input_images[str(ratio)[0:3]][ind][:, yy:yy + ps,
                xx:xx + ps, :]
        gt_patch = gt_images[ind][:, yy * 2:yy * 2 + ps * 2, xx * 2:xx
                                  * 2 + ps * 2, :]

        if np.random.randint(2) == 1:  # random flip for rows
            input_patch = np.flip(input_patch, axis=1)
            gt_patch = np.flip(gt_patch, axis=1)
        if np.random.randint(2) == 1:  # random flip for columns
            input_patch = np.flip(input_patch, axis=2)
            gt_patch = np.flip(gt_patch, axis=2)
        if np.random.randint(2) == 1:  # random transpose
            input_patch = np.transpose(input_patch, (0, 2, 1, 3))
            gt_patch = np.transpose(gt_patch, (0, 2, 1, 3))

        input_patch = np.minimum(input_patch, 1.0)

        (_, G_current, output) = sess.run([G_opt, G_loss, out_image],
                feed_dict={in_image: input_patch, gt_image: gt_patch,
                lr: learning_rate})
        output = np.minimum(np.maximum(output, 0), 1)
        g_loss[ind] = G_current

        print '%d %d Loss=%.3f Time=%.3f' % (epoch, cnt,
                np.mean(g_loss[np.where(g_loss)]), time.time() - st)

        if epoch % save_freq == 0:
            if not os.path.isdir(result_dir + '%04d' % epoch):
                os.makedirs(result_dir + '%04d' % epoch)

            temp = np.concatenate((gt_patch[0, :, :, :], output[0, :, :
                                  , :]), axis=1)
            scipy.misc.toimage(temp * 255, high=255, low=0, cmin=0,
                               cmax=255).save(result_dir
                    + '%04d/%05d_00_train_%d.jpg' % (epoch, train_id,
                    ratio))

    saver.save(sess, checkpoint_dir + 'model.ckpt')

In [0]:
from keras.layers import Input, Dense
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(784,))
tf.print(inputs)

<tf.Operation 'PrintV2' type=PrintV2>